<a href="https://colab.research.google.com/github/KwonYN/TIL/blob/master/PYTHON/TENSORFLOW2/03_02_01_char_rnn_7_kyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **실습 3-2 : Char-RNN**



## **Import Module**

In [0]:
%tensorflow_version 2.x

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

tf.__version__

TensorFlow 2.x selected.


'2.0.0'

## **DataSet**

### 학습할 문장 만들기

In [0]:
# 학습할 문장
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea." )
print ("FOLLOWING IS OUR TRAINING SEQUENCE:")
print (sentence)
print ("Length of 'test sentence' is %s" %len(sentence))

FOLLOWING IS OUR TRAINING SEQUENCE:
if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
Length of 'test sentence' is 180


### 입력 문자열과 타겟 문자 준비

In [0]:
# make charater dictionary 
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}
print ("CHARACTERS :", len(char_set))
print ("ㄴ", char_set)
print ("DICTIONARY :", len(char_dic))
print ('ㄴ', char_dic)

CHARACTERS : 25
ㄴ [',', 'b', 'y', ' ', 'c', 'h', 'u', 'd', 'r', 'i', 'w', 'o', '.', "'", 'a', 's', 'n', 'g', 't', 'f', 'l', 'm', 'k', 'e', 'p']
DICTIONARY : 25
ㄴ {',': 0, 'b': 1, 'y': 2, ' ': 3, 'c': 4, 'h': 5, 'u': 6, 'd': 7, 'r': 8, 'i': 9, 'w': 10, 'o': 11, '.': 12, "'": 13, 'a': 14, 's': 15, 'n': 16, 'g': 17, 't': 18, 'f': 19, 'l': 20, 'm': 21, 'k': 22, 'e': 23, 'p': 24}


In [0]:
data_dim    = len(char_set) # train data X:input
num_classes = len(char_set) # trian data Y:target
sequence_length = 10        # any arbitrary number
print ('data_dim : %d' %data_dim)
print ('num_classes : %d' %num_classes)

data_dim : 25
num_classes : 25


In [0]:
dataX = []  # input sequence list
dataY = []  # target sequence list 

# we will make 170 sequences 
for i in range(0, len(sentence) - sequence_length):
    # 10 characters = 1 training sequence
    x_str = sentence[i : i+sequence_length]
    y_str = sentence[i+1 : i+sequence_length+1]
    # convert x, y str to index(int)
    x_idx = [char_dic[c] for c in x_str] 
    y_idx = [char_dic[c] for c in y_str] 
    # append to dataset list
    dataX.append(x_idx)
    dataY.append(y_idx)

    # monitoring
    if i<5:
        print ("[%3d/%3d] [%s]=>[%s]" % (i, len(sentence), x_str, y_str))
        print ("%s%s=>%s" % (' '*10, x_idx, y_idx))

[  0/180] [if you wan]=>[f you want]
          [9, 19, 3, 2, 11, 6, 3, 10, 14, 16]=>[19, 3, 2, 11, 6, 3, 10, 14, 16, 18]
[  1/180] [f you want]=>[ you want ]
          [19, 3, 2, 11, 6, 3, 10, 14, 16, 18]=>[3, 2, 11, 6, 3, 10, 14, 16, 18, 3]
[  2/180] [ you want ]=>[you want t]
          [3, 2, 11, 6, 3, 10, 14, 16, 18, 3]=>[2, 11, 6, 3, 10, 14, 16, 18, 3, 18]
[  3/180] [you want t]=>[ou want to]
          [2, 11, 6, 3, 10, 14, 16, 18, 3, 18]=>[11, 6, 3, 10, 14, 16, 18, 3, 18, 11]
[  4/180] [ou want to]=>[u want to ]
          [11, 6, 3, 10, 14, 16, 18, 3, 18, 11]=>[6, 3, 10, 14, 16, 18, 3, 18, 11, 3]


### 입력문자 list를 3-dim로, 타겟문자 list를 2-dim로 변환  

In [0]:
# input tensor 생성 
X = np.array(dataX)                                      # ndarray(170,10)<-[170,10]
# convert list sequence to 3dim array(one-hot coding)
sequences = [tf.keras.utils.to_categorical(
                 x, num_classes = data_dim) for x in X]  # list[170,10,25]
X = np.array(sequences)                                  # ndarray(170,10,25)
print("shape of X :", X.shape)

# Target tensor 생성
#y = np.array(dataY)[:,-1]                                # ndarray(170,)<-[170,10]
y = np.array(dataY)                                       # ndarray(170,10)<-[170,10]
print("shape of y :", y.shape)

# convert the vector to 2-dim 
y = tf.keras.utils.to_categorical(
                  y, num_classes = data_dim)             # (170,25) -> (170,10,25)

print ("shape of y :", y.shape) # (170,10,25)
#print (y[0])    # t -> one-hot ,  (if you wan't')
print (y[0])

shape of X : (170, 10, 25)
shape of y : (170, 10)
shape of y : (170, 10, 25)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0.]]


## **Model**

### Define

In [0]:
# Sequenctial model define: LSTM + Dense
hidden_dim = 75 #150
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(hidden_dim,
          input_shape=(sequence_length,num_classes),
          return_sequences=True)) # (bs,10,hidden_dim)<-(bs,hidden_dim)
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(data_dim, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 75)            30300     
_________________________________________________________________
dense (Dense)                (None, 10, 25)            1900      
Total params: 32,200
Trainable params: 32,200
Non-trainable params: 0
_________________________________________________________________


### Compile

In [0]:
model.compile(loss ='categorical_crossentropy', # one-hot coding
              optimizer='adam', 
              metrics=['accuracy'])

### Fit

Epoch 1000/1000   
170/170 [==============================] - 0s 235us/sample - loss: 0.0012 - accuracy: 1.0000    
CPU times: user 48.3 s, sys: 6.54 s, total: 54.8 s    
Wall time: 47.8 s (@Notebook Setting/GPU)

In [0]:
%%time
model.fit(X,            # X.shape : (170, 10, 25)
          y,            # y.shape : (170, 10, 25)
          epochs=100)

Train on 170 samples
Epoch 1/100
170/170 [==============================] - 5s 29ms/sample - loss: 3.2095 - accuracy: 0.0582
Epoch 2/100
170/170 [==============================] - 0s 202us/sample - loss: 3.1749 - accuracy: 0.1771
Epoch 3/100
170/170 [==============================] - 0s 195us/sample - loss: 3.1332 - accuracy: 0.1988
Epoch 4/100
170/170 [==============================] - 0s 195us/sample - loss: 3.0725 - accuracy: 0.1959
Epoch 5/100
170/170 [==============================] - 0s 193us/sample - loss: 2.9893 - accuracy: 0.1947
Epoch 6/100
170/170 [==============================] - 0s 198us/sample - loss: 2.9544 - accuracy: 0.1965
Epoch 7/100
170/170 [==============================] - 0s 197us/sample - loss: 2.9198 - accuracy: 0.1971
Epoch 8/100
170/170 [==============================] - 0s 193us/sample - loss: 2.8970 - accuracy: 0.1982
Epoch 9/100
170/170 [==============================] - 0s 202us/sample - loss: 2.8739 - accuracy: 0.2012
Epoch 10/100
170/170 [=============

## **Generate Text:** analysis

### Related Module Import

### Define generate_seq function

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, char_dic, seq_length, seed_text, n_chars):
	in_text = seed_text									# seed text + generated text
	# generate a fixed number of characters
	for i in range(n_chars):
		# encode the characters as integers
		encoded = [char_dic[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# one hot encode
		encoded = tf.keras.utils.to_categorical(encoded, num_classes=len(char_dic))
		#encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		for char, index in char_dic.items():
			if index == yhat[0][-1]:
				break
		# append to input
		in_text += char
	return in_text

### Generate a sequence of characters

학습문장: if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

In [0]:
# test start of rhyme
print('Trigger characters: "want to bu "\nResult: \n"{}"'.format( 
      generate_seq(model, char_dic, sequence_length, 'want to bu', 200)))
# test mid-line
print('\nTrigger characters: "collect wo"\nResult: \n"{}"'.format(
      generate_seq(model, char_dic, sequence_length, 'collect wo', 200)))
# test mid-line
print('\nTrigger characters: "rather tea"\nResult: "{}"'.format(
      generate_seq(model, char_dic, sequence_length, 'rather tea', 200)))
# test not in original
print('\nTrigger characters: "aabbcr than"\nResult: "{}"'.format(
      generate_seq(model, char_dic, sequence_length, 'aabbcr than', 200)))
# test not in original
print('\nTrigger characters: "hidoyi"\nResult: "{}"'.format(
      generate_seq(model, char_dic, sequence_length, 'hidoyi', 200)))

Trigger characters: "want to bu "
Result: 
"want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the seasks and work, but rather teach them to"

Trigger characters: "collect wo"
Result: 
"collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the seasks and work, but rather teach them to long for the endless immensity of the seasks and work,"

Trigger characters: "rather tea"
Result: "rather teach them to long for the endless immensity of the seasks and work, but rather teach them to long for the endless immensity of the seasks and work, but rather teach them to long for the endless immensit"

Trigger characters: "aabbcr than"
Result: "aabbcr than them tasks and work, but rather teach them to long for the endless immensity of the seasks and work, but rather teach them to long for the endless immensity of the seasks and wo

In [0]:
#@title
print('\nTrigger characters: "aabbcr than"\nResult: "{}"'.format(
      generate_seq(model, char_dic, sequence_length, 'ports', 200)))


Trigger characters: "aabbcr than"
Result: "ports theenetter to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the seasks and work, but rather teach them to long for the endless immensit"


## **실습과제**

### 과제 1. 학습문장을 Keras Documents에서 2000자 정도 가져다 넣어서 실행해 보자('.'뒤에'\n'도 넣어주자) 
#### -- accuracy가 적당한가 확인하자.
#### -- 모델을 강화 하기 위해 `hidden_dim ='값을 150으로 높히고, 
#### -- `#`로 막혀 있는 `BatchNormalization()`을 동작시켜 보자
#### -- `Trigger characters`를 다양하게 바꿔보자 
#### -- 결과를 확인하고, 분석해 보자:
 


In [0]:
new_sentence = """When it comes to Donald Trump and Russia, everything is connected.
That’s the most important lesson from the two big events that played out Monday on Capitol Hill — the House Judiciary Committee’s hearings on President Trump’s impeachment and the release of the report on the origins of the F.B.I.’s investigation into ties between the Trump campaign and the Russian government.
One of these involved the 2016 election. The other involves the 2020 election. Both tell versions of the same story: Mr. Trump depends on, and welcomes, Russian interference to help him win the presidency. That was bad enough when he did it in 2016, openly calling for Russia to hack into his opponent’s emails — which Russians tried to do that same day. But he was only a candidate then. Now that Mr. Trump is president, he is wielding the immense powers of his office to achieve the same end.
That is precisely the type of abuse of power that the founders were most concerned about when they created the impeachment power, and it’s why Democratic leaders in the House are pressing ahead with such urgency on their inquiry. They are trying to ensure that the 2020 election, now less than a year away, is not corrupted by the president of the United States, acting in league with a foreign power. 'The integrity of our next election is at stake,' said Representative Jerry Nadler, the chairman of the Judiciary Committee. “Nothing could be more urgent.”
On Monday morning, lawyers for the Democrats on the House Judiciary and Intelligence committees presented the clearest and most comprehensive narrative yet of President Trump’s monthslong shakedown of the new Ukrainian president, Volodymyr Zelensky, for Mr. Trump’s personal political benefit. They explained in methodical detail how the president withheld a White House meeting and hundreds of millions of dollars in crucial, congressionally authorized military aid to Ukraine, all in an effort to get Mr. Zelensky to announce two investigations — one into Mr. Trump’s political rival, Joe Biden, and his son, Hunter, and another into Ukraine’s supposed interference in the 2016 election.
Who would benefit from these announcements? Mr. Trump, who believes his re-election prospects are threatened most by Mr. Biden, and Vladimir Putin, the president of Russia, who has been working for years to make Ukraine the fall guy for his own interference in the 2016 election. Mr. Putin has not fooled serious people, like those in the American intelligence community who determined that his government alone was responsible for meddling on Mr. Trump’s behalf. But he has fooled Republicans in Congress, who have degraded themselves and their offices by faithfully parroting Mr. Putin’s propaganda in the mainstream press.
House Republicans continued to do their best to obfuscate and misdirect, as they have done throughout the impeachment process. From the start of Monday’s hearing, they yelled and objected — an understandable strategy for people whose most honest defense of the president’s actions has been: 'Yeah, he did it. So what?' But this was not a performance for the history books; it was for the president himself, who registers only who is defending him and with how much fist-pounding rage.
The upshot is that Republicans, who once claimed to be the party of patriots and national security devotees, are on the record as having no problem with an American president using hundreds of millions of taxpayer dollars in an effort to induce foreign leaders to interfere in an election on his behalf. (Unless that president were a Democrat, of course.)
The emptiness of the Republicans’ case was exemplified in an interruption by Representative Mike Johnson of Louisiana, who stopped the morning’s proceedings after Barry Berke, the Judiciary Committee’s Democratic counsel, testified that President Trump had committed impeachable acts. 'The witness has used language which impugns the motives of the president and suggests he’s disloyal to his country, and those words should be stricken from the record and taken down,\" Mr. Johnson said. He was referring to a House rule that applies only to members of Congress, not witnesses. But consider the message he was sending: In the face of the most damning facts brought forward about a president in more than a generation, Mr. Johnson’s instinct was to say, in effect, How dare you speak the truth about our dear leader?
Speaking the truth, or accepting it when they hear it, is a skill that has become increasingly foreign to Republicans. Which brings us to the other big news of the day: the release of the 434-page report by the Justice Department’s inspector general, Michael Horowitz, into the basis for the F.B.I.’s investigation of the Trump campaign’s ties to Russia during the 2016 campaign. The evidence for these ties — like the evidence that Mr. Trump sought a bribe from the Ukrainian president — is indisputable. So Republicans have instead attacked the investigation itself, claiming that it was the fruit of illegal actions by Trump-hating F.B.I. agents.
Mr. Horowitz shot down that theory. The investigation was based on sufficient evidence of an ongoing national security threat, he found, and there was no evidence it was infected by anti-Trump political bias. In fact, Mr. Horowitz turned up evidence of agents who supported Mr. Trump. One wrote to another that he was \"so elated with the election,\" later explaining to the inspector general that he had opposed Hillary Clinton and “didn’t want a criminal to be in the White House.'
Still, Mr. Horowitz harshly criticized the F.B.I. over what he called \"many basic and fundamental errors\" it made during the investigation, including mistakes or omissions in important paperwork that “made it appear that the information supporting probable cause was stronger than was actually the case.\"
These are serious and in some cases alarming missteps and would be a concern in any case, let alone in one targeting people closely associated with a presidential campaign. But don’t be lulled into thinking Republicans care about surveillance overreach — just as they don’t care about Hillary Clinton’s emails or Joe Biden’s son. They’re simply casting about for anything that will protect their president from the scrutiny that he has invited upon himself since long before he was elected.
For his part, Mr. Trump claimed that the report was 'everything that a lot of people thought it would be, except far worse.'
This sort of brazen misrepresentation of facts is par for Mr. Trump’s course, and while it is sadly effective, it shouldn’t distract Americans from the emergency at hand: Whether Mr. Trump played footsie with Russians to win in 2016 or strong-armed a vulnerable new leader who is at war with Russia to win in 2020, he did exactly what the nation’s framers feared an unprincipled, power-hungry executive might do to hold onto his office. That’s why they put the impeachment power in the Constitution, and it’s why lawmakers who care about the Republic and its survival must use it.
"""
new_sentence = new_sentence.replace('.', '.\n')

print ("FOLLOWING IS OUR TRAINING SEQUENCE:")
print ("Length of 'test sentence' is %s" %len(new_sentence))

FOLLOWING IS OUR TRAINING SEQUENCE:
Length of 'test sentence' is 7112


### 입력 문자열과 타겟 문자 준비

In [0]:
# make charater dictionary 
new_char_set = list(set(new_sentence))
new_char_dic = {w: i for i, w in enumerate(new_char_set)}
print ("CHARACTERS :", len(new_char_set))
print ("ㄴ", new_char_set)
print ("DICTIONARY :", len(new_char_dic))
print ('ㄴ', new_char_dic)

CHARACTERS : 69
ㄴ [',', '—', 'O', 'A', '4', '3', 'W', '-', 'C', 'b', 'y', '’', ' ', 'M', 'c', '0', 'h', 'u', 'Y', 'd', 'S', 'v', '“', 'r', 'i', 'w', 'J', '1', 'o', '.', "'", 'R', 'a', 's', 'P', 'z', 'n', 'g', 'H', 't', '\n', 'N', 'q', '(', ')', 'T', 'L', '?', '"', '2', '”', ':', 'F', 'V', 'D', 'j', 'I', 'f', ';', 'm', 'l', 'B', 'k', 'e', '6', 'x', 'p', 'U', 'Z']
DICTIONARY : 69
ㄴ {',': 0, '—': 1, 'O': 2, 'A': 3, '4': 4, '3': 5, 'W': 6, '-': 7, 'C': 8, 'b': 9, 'y': 10, '’': 11, ' ': 12, 'M': 13, 'c': 14, '0': 15, 'h': 16, 'u': 17, 'Y': 18, 'd': 19, 'S': 20, 'v': 21, '“': 22, 'r': 23, 'i': 24, 'w': 25, 'J': 26, '1': 27, 'o': 28, '.': 29, "'": 30, 'R': 31, 'a': 32, 's': 33, 'P': 34, 'z': 35, 'n': 36, 'g': 37, 'H': 38, 't': 39, '\n': 40, 'N': 41, 'q': 42, '(': 43, ')': 44, 'T': 45, 'L': 46, '?': 47, '"': 48, '2': 49, '”': 50, ':': 51, 'F': 52, 'V': 53, 'D': 54, 'j': 55, 'I': 56, 'f': 57, ';': 58, 'm': 59, 'l': 60, 'B': 61, 'k': 62, 'e': 63, '6': 64, 'x': 65, 'p': 66, 'U': 67, 'Z': 68}


In [0]:
new_data_dim    = len(new_char_set) # train data X:input
new_num_classes = len(new_char_set) # trian data Y:target
new_sequence_length = 10        # any arbitrary number
print ('data_dim : %d' %new_data_dim)
print ('num_classes : %d' %new_num_classes)

data_dim : 69
num_classes : 69


In [0]:
dataX = []  # input sequence list
dataY = []  # target sequence list 

# we will make 170 sequences 
for i in range(0, len(new_sentence) - new_sequence_length):
    # 10 characters = 1 training sequence
    x_str = new_sentence[i : i+new_sequence_length]
    y_str = new_sentence[i+1 : i+new_sequence_length+1]
    # convert x, y str to index(int)
    x_idx = [new_char_dic[c] for c in x_str] 
    y_idx = [new_char_dic[c] for c in y_str] 
    # append to dataset list
    dataX.append(x_idx)
    dataY.append(y_idx)

    # monitoring
    if i<5:
        print ("[%3d/%3d] [%s]=>[%s]" % (i, len(new_sentence), x_str, y_str))
        print ("%s%s=>%s" % (' '*10, x_idx, y_idx))

[  0/7112] [When it co]=>[hen it com]
          [6, 16, 63, 36, 12, 24, 39, 12, 14, 28]=>[16, 63, 36, 12, 24, 39, 12, 14, 28, 59]
[  1/7112] [hen it com]=>[en it come]
          [16, 63, 36, 12, 24, 39, 12, 14, 28, 59]=>[63, 36, 12, 24, 39, 12, 14, 28, 59, 63]
[  2/7112] [en it come]=>[n it comes]
          [63, 36, 12, 24, 39, 12, 14, 28, 59, 63]=>[36, 12, 24, 39, 12, 14, 28, 59, 63, 33]
[  3/7112] [n it comes]=>[ it comes ]
          [36, 12, 24, 39, 12, 14, 28, 59, 63, 33]=>[12, 24, 39, 12, 14, 28, 59, 63, 33, 12]
[  4/7112] [ it comes ]=>[it comes t]
          [12, 24, 39, 12, 14, 28, 59, 63, 33, 12]=>[24, 39, 12, 14, 28, 59, 63, 33, 12, 39]


### 입력문자 list를 3-dim로, 타겟문자 list를 2-dim로 변환  

In [0]:
# input tensor 생성 
X = np.array(dataX)                                      # ndarray(170,10)<-[170,10]
# convert list sequence to 3dim array(one-hot coding)
sequences = [tf.keras.utils.to_categorical(
                 x, num_classes = new_data_dim) for x in X]  # list[170,10,25]
X = np.array(sequences)                                  # ndarray(170,10,25)
print("shape of X :", X.shape)

# Target tensor 생성
#y = np.array(dataY)[:,-1]                                # ndarray(170,)<-[170,10]
y = np.array(dataY)                                       # ndarray(170,10)<-[170,10]
print("shape of y :", y.shape)

# convert the vector to 2-dim 
y = tf.keras.utils.to_categorical(
                  y, num_classes = new_data_dim)             # (170,25) -> (170,10,25)

print ("shape of y :", y.shape)# (170,10,25)
#print (y[0])    # t -> one-hot ,  (if you wan't')

shape of X : (7102, 10, 69)
shape of y : (7102, 10)
shape of y : (7102, 10, 69)


## **Model**

### Define

In [0]:
# Sequenctial model define: LSTM + Dense
hidden_dim = 150
model_2=tf.keras.models.Sequential(name='NewYorkTimes')
model_2.add(tf.keras.layers.LSTM(hidden_dim,
          input_shape=(new_sequence_length,new_num_classes),
          return_sequences=True)) # (bs,10,hidden_dim)<-(bs,hidden_dim)
model.add(tf.keras.layers.BatchNormalization())
model_2.add(tf.keras.layers.Dense(new_data_dim, activation='softmax'))

model_2.summary()

Model: "NewYorkTimes"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10, 150)           132000    
_________________________________________________________________
dense_2 (Dense)              (None, 10, 69)            10419     
Total params: 142,419
Trainable params: 142,419
Non-trainable params: 0
_________________________________________________________________


### Compile

In [0]:
model_2.compile(loss ='categorical_crossentropy', # one-hot coding
              optimizer='adam', 
              metrics=['accuracy'])

### Fit

Epoch 1000/1000   
170/170 [==============================] - 0s 235us/sample - loss: 0.0012 - accuracy: 1.0000    
CPU times: user 48.3 s, sys: 6.54 s, total: 54.8 s    
Wall time: 47.8 s (@Notebook Setting/GPU)

In [0]:
%%time
model_2.fit(X,            # X.shape : (170, 10, 25)
            y,            # y.shape : (170, 10, 25)
            epochs=100)

Train on 7102 samples
Epoch 1/100
7102/7102 [==============================] - 4s 520us/sample - loss: 3.2367 - accuracy: 0.1755
Epoch 2/100
7102/7102 [==============================] - 1s 163us/sample - loss: 2.6099 - accuracy: 0.3092
Epoch 3/100
7102/7102 [==============================] - 1s 159us/sample - loss: 2.2967 - accuracy: 0.3639
Epoch 4/100
7102/7102 [==============================] - 1s 160us/sample - loss: 2.1334 - accuracy: 0.3948
Epoch 5/100
7102/7102 [==============================] - 1s 162us/sample - loss: 2.0139 - accuracy: 0.4236
Epoch 6/100
7102/7102 [==============================] - 1s 163us/sample - loss: 1.9125 - accuracy: 0.4487
Epoch 7/100
7102/7102 [==============================] - 1s 163us/sample - loss: 1.8202 - accuracy: 0.4696
Epoch 8/100
7102/7102 [==============================] - 1s 159us/sample - loss: 1.7298 - accuracy: 0.4953
Epoch 9/100
7102/7102 [==============================] - 1s 162us/sample - loss: 1.6471 - accuracy: 0.5166
Epoch 10/100
71

## **Generate Text:** analysis

### Related Module Import

### Define generate_seq function

In [0]:
print('Trigger characters: "When it co "\nResult: \n"{}"'.format( 
      generate_seq(model_2, new_char_dic, new_sequence_length, 'When it co', 200)))

print('\nTrigger characters: "their best"\nResult: \n"{}"'.format(
      generate_seq(model_2, new_char_dic, new_sequence_length, 'their best', 200)))

print('\nTrigger characters: "in an inte"\nResult: "{}"'.format(
      generate_seq(model_2, new_char_dic, new_sequence_length, 'in an inte', 200)))

print('\nTrigger characters: "hello "\nResult: "{}"'.format(
      generate_seq(model_2, new_char_dic, new_sequence_length, 'hello ', 200)))

Trigger characters: "When it co "
Result: 
"When it comes to Donald Trump and Russia, everything is connected.

That’s the most damning facts brought forward about a president were a Democrat, of course.
)
The emptiness of the Republicans continued to do"

Trigger characters: "their best"
Result: 
"their best to obfuscate and misdirect, as they have done throughout the impeachment and the release of the report was 'everything is connected.

That’s the most damning facts brought forward about a president w"

Trigger characters: "in an inte"
Result: "in an interruption by Representative Jerry Nadler, the chairman of the Judiciary Committee’s Democratic counsel, testified that President Trump’s impeachment and the release of the report was 'everything is con"

Trigger characters: "hello "
Result: "hello actory bas baser that the founders were most concerned about when they hear it, is a skill that has become increasingly foreign to Republicans continued to do their best to obfuscate and m

In [0]:
print('\nTrigger characters: "kwonyo"\nResult: "{}"'.format(
      generate_seq(model_2, new_char_dic, new_sequence_length, 'kwonyo', 200)))


Trigger characters: "kwonyo"
Result: "kwonyon and “didn’t want a criminal to be in the White House.
'
Still, Mr.
 Horowitz harshly criticized the F.
B.
I.
’s investigation was based on sufficient evidence of agents who supported Mr.
 Trump is p"


In [0]:
print('\nTrigger characters: "kwonyo"\nResult: "{}"'.format(
      generate_seq(model_2, new_char_dic, new_sequence_length, 'kwonyo', 50)))


Trigger characters: "kwonyo"
Result: "kwonyon and “didn’t want a criminal to be in the White H"


In [0]:
print('\nTrigger characters: "kwonyo"\nResult: "{}"'.format(
      generate_seq(model_2, new_char_dic, new_sequence_length, 'kwonyo', 10)))


Trigger characters: "kwonyo"
Result: "kwonyon and “did"
